## 4.3.3. Ewaluacja wyników segmentacji

#### Importy

In [ ]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path

#### Funkcja `load_image`
Funkcja odpowiedzialna za wczytanie obrazu i normalizacja pikseli do [0,1].

In [3]:
def load_image(path):
    img = cv2.imread(str(path))
    if img is None:
        print("Nie można wczytać obrazu")
        return None
    return img.astype(np.float32) / 255.0

#### Funkcja `load_mask`
Funkcja odpowiedziana za wczytanie obrazu maski i zastosowanie label smoothing który poprawia stabilność uczenia i ułatwia generalizację modelu

In [4]:
def load_mask(path, smooth=0.1):
    mask = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print("Nie można wczytać maski")
        return None
    m = mask.astype(np.float32) / 255.0
    
    m = m * (1 - smooth) + (smooth / 2)
    return np.expand_dims(m, axis=-1)

#### Funkcja `iou_metric`
Funkcja wyliczająca miarę IoU

In [5]:
def iou_metric(y_true, y_pred, smooth=1e-6):
    y_pred = tf.round(y_pred)
    inter = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - inter
    return (inter + smooth) / (union + smooth)

#### Funkcja `dice_coef`
Funkcja wylicająca miarę DICE

In [6]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_pred = tf.round(y_pred)
    inter = tf.reduce_sum(y_true * y_pred)
    return (2 * inter + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

#### Funkcja `recall_metric`
Funkcja wyliczająca miarę recall

In [7]:
def recall_metric(y_true, y_pred, smooth=1e-6):
    y_pred = tf.round(y_pred)
    tp = tf.reduce_sum(y_true * y_pred)
    return (tp + smooth) / (tf.reduce_sum(y_true) + smooth)

#### Funkcja `bce_dice_loss`
Funkcja obliczająca stratę

In [8]:
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = 1 - dice_coef(y_true, y_pred)
    return 0.7 * bce + 0.3 * dice

#### Załadowanie modelu

In [10]:
model = tf.keras.models.load_model(
    'unet_best_model.h5',
    custom_objects={
        'bce_dice_loss': bce_dice_loss,
        'iou_metric': iou_metric,
        'dice_coef': dice_coef,
        'recall_metric': recall_metric
    },
    compile=False,
)

#### Przygotowanie zbioru testowego

In [ ]:
test_pairs = []
for img_path in sorted(Path("./processed/images_to_train/u-net/test/images").glob("*.*")):
    base = img_path.stem
    mask_path = Path("./processed/images_to_train/u-net/test/masks") / f"{base}.jpg"
    if not mask_path.exists():
        continue
    test_pairs.append((img_path, mask_path))


#### Wyliczenie metryk

In [ ]:
total_iou, total_dice, total_recall = 0, 0, 0

for img_path, mask_path in test_pairs:
    img = load_image(img_path)
    if img is None:
        continue

    mask = load_mask(mask_path)
    if mask is None:
        continue

    img_input = np.expand_dims(img, axis=0)
    pred = model.predict(img_input, verbose=0)[0, :, :, 0]

    pred_mask = np.where(pred > 0.5, 1.0, 0.0)
    mask_bin = np.where(mask[:, :, 0] > 0.5, 1.0, 0.0)

    iou = iou_metric(mask_bin, pred_mask).numpy()
    dice = dice_coef(mask_bin, pred_mask).numpy()
    recall = recall_metric(mask_bin, pred_mask).numpy()

    total_iou += iou
    total_dice += dice
    total_recall += recall

## Wyniki
Prezentazja wyników i zapis do pliku

In [ ]:
n = len(test_pairs)
print(f"Test set (n={n}):")
print(f"IoU: {total_iou / n:.4f}")
print(f"Dice: {total_dice / n:.4f}")
print(f"Recall: {total_recall / n:.4f}")

df = pd.DataFrame({
    'Test set size': [n],
    'IoU': [total_recall / n],
    'Dice': [total_recall / n],
    'Recall': [total_recall / n]
})

df.to_csv('segmentation_test_results.csv', index=False)